In [1]:
import os
import logging
import random
import gc
import time
import cv2
import math
import warnings
from pathlib import Path

import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
import librosa

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader

import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm

import timm

warnings.filterwarnings("ignore")
logging.basicConfig(level=logging.ERROR)

In [2]:
cfg8 = {'seed': 10888,
 'debug': False,
 'print_freq': 100,
 'num_workers': 4,
 'stage': 'train_bce',
 'OUTPUT_DIR': './',
 'train_datadir': '../train_audio',
 'train_csv': 'train_meta.csv',
 'label_csv': 'label_meta.csv',
 'test_soundscapes': '../test_soundscapes',
 'submission_csv': 'sample_submission.csv',
 'taxonomy_csv': '../taxonomy.csv',
 'class_sample_count': 'class_sample_count.pkl',
 'model_name': 'seresnext26t_32x4d',
 'pretrained': True,
 'in_channels': 1,
 'img_size': 384,
 'target_duration': 5,
 'SR': 32000,
 'n_fft': 2048,
 'n_mels': 256,
 'f_min': 20,
 'f_max': 16000,
 'hop_length': 417,
 'device': 'cuda',
 'epochs': 30,
 'batch_size': 32,
 'criterion': 'BECFocal',
 'use_weights': False,
 'n_fold': 4,
 'selected_folds': [0, 1, 2, 3],
 'optimizer': 'AdamW',
 'lr': 0.002,
 'weight_decay': 0.0001,
 'scheduler': 'CosineAnnealingLR',
 'min_lr': 1e-06,
 'warmup_epo': 3,
 'T_max': 30,
 'mixup_prob': 0.1,
 'mixup_double': 1.0,
 'mix_beta': 5,
 'mix_beta2': 2,
 'mixup2_prob': 0.7,
 'cutmix_beta': 1.5,
 'cutmix_prob': 0.5,
 'sumix_max_percent': 1.0,
 'sumix_min_percent': 0.3,
 'mixup': False,
 'mixup2': True,
 'cutmix': False,
 'sumix': True,
 'normal': 80,
 'valid_duration': 5,
 'duration_train': 5,
 'infer_duration': 5,
 'primary_label_col': 'primary_label',
 'secondary_labels_col': 'secondary_labels',
 'class_exponent_weight': -0.5,
 'resume': None}

In [3]:
def get_optimizer(model, cfg):
  
    if cfg.optimizer == 'Adam':
        optimizer = optim.Adam(
            model.parameters(),
            lr=cfg.lr,
            weight_decay=cfg.weight_decay
        )
    elif cfg.optimizer == 'AdamW':
        optimizer = optim.AdamW(
            model.parameters(),
            lr=cfg.lr,
            weight_decay=cfg.weight_decay
        )
    elif cfg.optimizer == 'SGD':
        optimizer = optim.SGD(
            model.parameters(),
            lr=cfg.lr,
            momentum=0.9,
            weight_decay=cfg.weight_decay
        )
    else:
        raise NotImplementedError(f"Optimizer {cfg.optimizer} not implemented")
        
    return optimizer

def get_scheduler(optimizer, cfg):
   
    if cfg.scheduler == 'CosineAnnealingLR':
        scheduler = lr_scheduler.CosineAnnealingLR(
            optimizer,
            T_max=cfg.T_max,
            eta_min=cfg.min_lr
        )
    elif cfg.scheduler == 'ReduceLROnPlateau':
        scheduler = lr_scheduler.ReduceLROnPlateau(
            optimizer,
            mode='min',
            factor=0.5,
            patience=2,
            min_lr=cfg.min_lr,
            verbose=True
        )
    elif cfg.scheduler == 'StepLR':
        scheduler = lr_scheduler.StepLR(
            optimizer,
            step_size=cfg.epochs // 3,
            gamma=0.5
        )
    elif cfg.scheduler == 'OneCycleLR':
        scheduler = None  
    else:
        scheduler = None
        
    return scheduler

import torchvision
class FocalLossBCE(torch.nn.Module):
    def __init__(
            self,
            alpha: float = 0.25,
            gamma: float = 2.1,
            reduction: str = "mean",
            bce_weight: float = 0.61,
            focal_weight: float = 1.39,
    ):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction
        self.bce = torch.nn.BCEWithLogitsLoss(reduction=reduction)
        self.bce_weight = bce_weight
        self.focal_weight = focal_weight

    def forward(self, logits, targets):
        focall_loss = torchvision.ops.focal_loss.sigmoid_focal_loss(
            inputs=logits,
            targets=targets,
            alpha=self.alpha,
            gamma=self.gamma,
            reduction=self.reduction,
        )
        bce_loss = self.bce(logits, targets)
        return self.bce_weight * bce_loss + self.focal_weight * focall_loss

def get_criterion(cfg):
    return FocalLossBCE()

In [4]:
def train_one_epoch(model, loader, optimizer, criterion, device, scheduler=None):
    
    model.train()
    losses = []
    all_targets = []
    all_outputs = []
    
    pbar = tqdm(enumerate(loader), total=len(loader), desc="Training")
    
    for step, batch in pbar:
    
        if True:
            batch = [i.to(device) for i in batch]
            optimizer.zero_grad()
            output,y,w = model(batch)
            
            loss = criterion(output,y)
            loss.backward()
            optimizer.step()
            
        losses.append(loss if isinstance(loss, float) else loss.item())
        
        pbar.set_postfix({
            'train_loss': np.mean(losses[-10:]) if losses else 0,
            'lr': optimizer.param_groups[0]['lr']
        })
    
    auc = 0.0
    avg_loss = np.mean(losses)
    
    return avg_loss, auc

In [5]:
import argparse
import importlib
from modules.preprocess import preprocess,prepare_cfg
from modules.dataset import get_train_dataloader
from modules.model import load_model
import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import ModelCheckpoint, BackboneFinetuning, EarlyStopping
import torch
import os
import gc
import json

def make_parser():
    parser = argparse.ArgumentParser(description='parser')
    parser.add_argument('--stage', choices=["pretrain_ce","pretrain_bce","train_ce","train_bce","finetune"])
    parser.add_argument('--model_name', choices=["sed_v2s",'sed_b3ns','sed_seresnext26t','cnn_v2s','cnn_resnet34d','cnn_b3ns','cnn_b0ns'])
    parser.add_argument('--use_pseudo', action='store_true')
    return parser


def main():
    cfg = importlib.import_module(f'configs.sed_seresnext26t').basic_cfg
    for key, value in cfg8.items():
        setattr(cfg, key, value)

    taxonomy_df = pd.read_csv(cfg8["taxonomy_csv"])
    species_ids = taxonomy_df['primary_label'].tolist()
    setattr(cfg, "bird_cols_train", species_ids)
    cfg = prepare_cfg(cfg,cfg.stage)
    
    pl.seed_everything(cfg.seed, workers=True)
    

    df_train, df_valid, df_label_train, df_label_valid, sample_weight, transforms = preprocess(cfg)
    
    pseudo = None
    dl_train, dl_val, ds_train, ds_val = get_train_dataloader(
        df_train,
        df_valid,
        df_label_train,
        df_label_valid,
        sample_weight,
        cfg,
        pseudo,
        None
    )

    model = load_model(cfg,cfg.stage)
    model = model.to(cfg.device)
    optimizer = get_optimizer(model, cfg)
    scheduler = get_scheduler(optimizer, cfg)
    criterion = get_criterion(cfg)
    for epoch in range(cfg.epochs):
        print(f"\nEpoch {epoch+1}/{cfg.epochs}")
            
        train_loss, train_auc = train_one_epoch(
            model, 
            dl_train, 
            optimizer, 
            criterion, 
            cfg.device,
            None
        )
        scheduler.step()
        print(f"Train Loss: {train_loss:.4f}, Train AUC: {train_auc:.4f}")
        
        torch.save({
            'model_state_dict': model.state_dict(),
            'cfg': cfg8
        }, f"model_fold_{epoch+1}_{train_loss:.4f}.pth")

    del model, optimizer, scheduler, dl_train
    gc.collect()
    torch.cuda.empty_cache()
    return

main()


INFO:lightning_fabric.utilities.seed:Seed set to 10557



Epoch 1/30


Training:   0%|          | 0/893 [00:00<?, ?it/s]

Train Loss: 0.0255, Train AUC: 0.0000

Epoch 2/30


Training:   0%|          | 0/893 [00:00<?, ?it/s]

...

Train Loss: 0.0100, Train AUC: 0.0000

Epoch 29/30


Training:   0%|          | 0/893 [00:00<?, ?it/s]

Train Loss: 0.0099, Train AUC: 0.0000

Epoch 30/30


Training:   0%|          | 0/893 [00:01<?, ?it/s]

Train Loss: 0.0098, Train AUC: 0.0000
